In [ ]:
# --- CELL 1: SETUP ENVIRONMENT ---
import os
from google.colab import drive

# 1. Mount Drive (Your Data)
drive.mount('/content/drive')

# 2. Sync Code from GitHub
REPO_URL = "https://github.com/brauner68/gm_proj.git"
REPO_NAME = "gm_proj"

if not os.path.exists(f"/content/{REPO_NAME}"):
    print(f"⬇️ Cloning {REPO_NAME}...")
    !git clone {REPO_URL}
else:
    print(f"🔄 Repo exists. Pulling latest changes...")
    %cd /content/{REPO_NAME}
    !git pull

# 3. Enter Project Folder
%cd /content/{REPO_NAME}

# 4. Install Dependencies (The Magic Step)
# This reads your requirements.txt and installs everything at once
print("📦 Installing libraries from requirements.txt...")
!pip install -q -r requirements.txt

print("\n✅ SETUP COMPLETE! You are ready to work.")